In [37]:
data = pd.DataFrame([[0, 0, 1, 0, 0], 
                     [1, 0, 0, 1, 0], 
                     [1, 1, 0, 0, 1], 
                     [0, 0, 1, 0, 0], 
                     [0, 0, 1, 0, 0], 
                     [0, 1, 0, 0, 1]])
data.to_csv("data.csv", index=False, header=False)

In [39]:
import tensorflow as tf
import numpy as np

data = np.loadtxt("data.csv", delimiter=',', 
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

In [41]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [44]:
X = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [45]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [58]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())
    
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, y: y_data})
    
    print("Step: {}".format(sess.run(global_step)), 
          "Cost: {:.3f}".format(sess.run(cost, feed_dict={X: x_data, y: y_data})))
    
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

prediction = tf.argmax(model, axis=1)
target = tf.argmax(y, axis=1)
print("Prediction: ", sess.run(prediction, feed_dict={X: x_data}))
print("Actual: ", sess.run(target, feed_dict={y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("Accuracy: {:.2f}".format(sess.run(accuracy * 100, feed_dict={X: x_data,
                                                                    y: y_data})))

INFO:tensorflow:Restoring parameters from ./model/dnn.ckpt-20
Step: 21 Cost: 0.657
Step: 22 Cost: 0.643
Prediction:  [0 1 2 0 0 2]
Actual:  [0 1 2 0 0 2]
Accuracy: 100.00
